In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train=pd.read_csv('/content/train.csv')
df_games=pd.read_csv('/content//games.csv')
df_turns=pd.read_csv('/content/turns.csv')
df_test=pd.read_csv('/content//test.csv')
print(len(df_train),len(df_test),len(df_games),len(df_turns))
print(len(df_train.game_id.unique()),len(df_test.game_id.unique()),len(df_games.game_id.unique()),len(df_turns.game_id.unique()))

100820 44726 72773 2005498
50410 22363 72773 72773


In [ ]:
df_train['has_won']=-1
for ind in df_train.index:
    if(ind%2==0):
        if(df_train['score'][ind]>df_train['score'][ind+1]):
            df_train['has_won'][ind]=1
        else:
            df_train['has_won'][ind]=0
    else:
        if(df_train['score'][ind]>df_train['score'][ind-1]):
            df_train['has_won'][ind]=1
        else:
            df_train['has_won'][ind]=0

<ipython-input-19-9e7d04eca030>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['has_won'][ind]=0
<ipython-input-19-9e7d04eca030>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['has_won'][ind]=1
<ipython-input-19-9e7d04eca030>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['has_won'][ind]=1
<ipython-input-19-9e7d04eca030>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [ ]:
df_test['has_won']=-1
for ind in df_test.index:
    if(ind%2==0):
        if(df_test['score'][ind]>df_test['score'][ind+1]):
            df_test['has_won'][ind]=1
        else:
            df_test['has_won'][ind]=0
    else:
        if(df_test['score'][ind]>df_test['score'][ind-1]):
            df_test['has_won'][ind]=1
        else:
            df_test['has_won'][ind]=0

<ipython-input-20-0bf4b459729b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['has_won'][ind]=1
<ipython-input-20-0bf4b459729b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['has_won'][ind]=0
<ipython-input-20-0bf4b459729b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['has_won'][ind]=0
<ipython-input-20-0bf4b459729b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [ ]:
def first(row):
    if(row['nickname']==row['first']):
        return 1
    else:
        return 0

In [ ]:
df_t=df_train.set_index('game_id').join(df_games.set_index('game_id'))
df_t['has_begin'] = df_t.apply(lambda row: first(row), axis=1)
df_t=df_t.drop(['winner','first'],axis=1)

In [ ]:
df_tt=df_test.set_index('game_id').join(df_games.set_index('game_id'))
df_tt['has_begin'] = df_tt.apply(lambda row: first(row), axis=1)
df_tt=df_tt.drop(['winner','first'],axis=1)

In [ ]:
#Dont execute 
#checkpoint saving for preprocessed data
df_t.to_csv('train_final1.csv')
df_tt.to_csv('test_final1.csv')

In [ ]:
features=df_turns[['game_id','nickname','points','turn_type']]
features=features.dropna()
features['keys']=features['game_id'].apply(str)
features['keys']=features['keys']+'_'+features['nickname']
dummies = pd.get_dummies(features.turn_type)
features = pd.concat([features, dummies], axis='columns')
features=features.drop(['turn_type'],axis=1)

In [ ]:
train_copy = features.groupby(['keys']).aggregate({'game_id':'max','nickname':'max','points': 'mean','Challenge' : 'sum','End' : 'sum', 'Exchange':'sum','Pass':'sum','Play':'sum','Six-Zero Rule':'sum','Timeout':'sum' })
train_copy.reset_index(drop=True)
train_copy.head(10)

,game_id,nickname,points,Challenge,End,Exchange,Pass,Play,Six-Zero Rule,Timeout
keys,,,,,,,,,,
10000_STEEBot,10000,STEEBot,27.400000,0,1,0,0,14,0,0
10000_barikayak,10000,barikayak,31.769231,0,0,0,0,13,0,0
10001_STEEBot,10001,STEEBot,33.153846,0,1,1,0,11,0,0
10001_marvin,10001,marvin,32.000000,0,0,0,0,11,0,1
10002_STEEBot,10002,STEEBot,35.294118,1,1,0,0,15,0,0
10002_hulkbuster,10002,hulkbuster,13.352941,0,0,2,0,15,0,0
10003_Anita,10003,Anita,19.250000,0,1,1,0,14,0,0
10003_STEEBot,10003,STEEBot,34.538462,0,0,0,0,13,0,0
10004_HastyBot,10004,HastyBot,37.181818,0,0,0,0,11,0,0


In [ ]:
df_t['rating_mode'].replace(['CASUAL', 'RATED'],[0, 1], inplace=True)
dummies1 = pd.get_dummies(df_t.game_end_reason)
df_t = pd.concat([df_t, dummies1], axis='columns')
df_t=df_t.drop(['game_end_reason'],axis=1)
dummies2 = pd.get_dummies(df_t.lexicon)
df_t = pd.concat([df_t, dummies2], axis='columns')
df_t=df_t.drop(['lexicon'],axis=1)
df_t = pd.merge(df_t, train_copy, how='left',left_on=['game_id','nickname'],right_on=['game_id','nickname'])
df_t['time_control_name'].replace(['regular', 'rapid', 'blitz', 'ultrablitz'],[0, 1,2,3], inplace=True)
dummies3 = pd.get_dummies(df_t.time_control_name)
df_t = pd.concat([df_t, dummies3], axis='columns')
df_t=df_t.drop(['time_control_name','created_at'],axis=1)
df_t

In [ ]:
df_tt['rating_mode'].replace(['CASUAL', 'RATED'],[0, 1], inplace=True)
dummies = pd.get_dummies(df_tt.game_end_reason)
df_tt = pd.concat([df_tt, dummies], axis='columns')
df_tt=df_tt.drop(['game_end_reason'],axis=1)
dummies = pd.get_dummies(df_tt.lexicon)
df_tt = pd.concat([df_tt, dummies], axis='columns')
df_tt=df_tt.drop(['lexicon'],axis=1)

df_tt = pd.merge(df_tt, train_copy, how='left',left_on=['game_id','nickname'],right_on=['game_id','nickname'])
dummies = pd.get_dummies(df_tt.time_control_name)
df_tt = pd.concat([df_tt, dummies], axis='columns')
df_tt=df_tt.drop(['time_control_name','created_at'],axis=1)
df_tt['NSWL20']=0
df_tt=df_tt.reindex(columns=df_t.columns)
df_tt


,game_id,nickname,score,rating,has_won,time_control_name,lexicon,initial_time_seconds,increment_seconds,rating_mode,...,STANDARD,TIME,points,Challenge,End,Exchange,Pass,Play,Six-Zero Rule,Timeout
0,2,Super,488,NaN,1,0,1,3600,0,1,...,1,0,34.857143,0,1,1,1,11,0,0
1,2,BetterBot,401,2000.0,0,0,1,3600,0,1,...,1,0,33.416667,0,0,0,0,12,0,0
2,7,STEEBot,377,2082.0,0,0,1,1260,0,1,...,1,0,26.928571,0,1,0,0,13,0,0
3,7,Priya1,379,NaN,1,0,1,1260,0,1,...,1,0,29.153846,0,0,1,0,12,0,0
4,11,STEEBot,334,1829.0,0,0,0,1200,0,0,...,1,0,22.266667,0,1,1,0,13,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44721,72762,kyjo55555,367,NaN,1,0,0,3600,0,0,...,1,0,26.214286,0,0,1,0,13,0,0
44722,72768,HastyBot,524,2356.0,1,0,1,3600,0,1,...,1,0,47.636364,0,0,0,0,11,0,0
44723,72768,Maximilian,357,NaN,0,0,1,3600,0,1,...,1,0,29.750000,0,1,0,0,11,0,0
44724,72769,STEEBot,626,2110.0,1,2,1,180,0,1,...,1,0,48.153846,0,0,0,0,13,0,0


In [ ]:
df_tt1=df_tt.dropna()
df_tt2=df_tt[df_tt.isna().any(axis=1)]

In [ ]:

y=df_t['rating']
x=df_t.drop(['rating','game_id','nickname','initial_time_seconds', 'increment_seconds'],axis=1)
y_test=df_tt1['rating']
x_test=df_tt1.drop(['rating','game_id','nickname','initial_time_seconds', 'increment_seconds'],axis=1)

In [ ]:
x.columns

Index(['score', 'has_won', 'rating_mode', 'max_overtime_minutes',
       'game_duration_seconds', 'has_begin', 'CONSECUTIVE_ZEROES', 'RESIGNED',
       'STANDARD', 'TIME', 'CSW21', 'ECWL', 'NSWL20', 'NWL20', 'points',
       'Challenge', 'End', 'Exchange', 'Pass', 'Play', 'Six-Zero Rule',
       'Timeout', 'blitz', 'rapid', 'regular', 'ultrablitz'],
      dtype='object')

In [ ]:
np.all(np.isfinite(x))

True

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols=x.columns
x1=pd.DataFrame(scaler.fit_transform(x),columns=cols)
x_test1=pd.DataFrame(scaler.transform(x_test),columns=cols)

In [ ]:
np.all(np.isfinite(x1))

True

In [ ]:
np.all(np.isfinite(y))

True

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 30)
clf.fit(x1, y)
y_pred_train_bot_random_forest = clf.predict(x1)
rmse_train_bot_randomforest=mean_squared_error(y,y_pred_train_bot_random_forest,squared=False)
print("RMSE of train data for using random forest is : ",rmse_train_bot_randomforest )

RMSE of train data for using random forest is :  3.7275679663636376


In [ ]:
y_pred = clf.predict(x_test1)
rmse_test=mean_squared_error(y_test,y_pred,squared=False)
print("RMSE of test data for using random forest is : ",rmse_test )

RMSE of test data for using random forest is :  185.83130384156743


In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x1, y)
y_pred_reg=reg.predict(x1)
print("RMSE of train data for using random forest is : ",mean_squared_error(y,y_pred_reg,squared=False) )

RMSE of train data for using random forest is :  138.7164191810331


In [ ]:
y_pred_reg1=reg.predict(x_test1)
print("RMSE of train data for using random forest is : ",mean_squared_error(y_test,y_pred_reg1,squared=False) )

RMSE of train data for using random forest is :  129.07583149671606


In [ ]:
from sklearn.neural_network import MLPRegressor
mlp_reg = MLPRegressor(hidden_layer_sizes=(200,100,50),
                       max_iter = 200,activation = 'relu',
                       solver = 'adam')

mlp_reg.fit(x1,y)
preds= mlp_reg.predict(x_test1)
realVals=y_test
rmse_test=mean_squared_error(realVals, preds,squared=False)
print("RMSE of test data : ",rmse_test)

RMSE of test data :  131.3812068150747


In [ ]:
y_pred_nn=mlp_reg.predict(x1)
print("RMSE train data:",mean_squared_error(y,y_pred_nn,squared=False))

RMSE train data: 122.72423744986038


In [ ]:
from sklearn.linear_model import BayesianRidge
reglog = BayesianRidge().fit(x1, y)

In [ ]:
y_pred_bas=reglog.predict(x1)
print("RMSE train data:",mean_squared_error(y,y_pred_bas,squared=False))

RMSE train data: 138.71565114345663


In [ ]:
y_pred_bas

array([1654.44684728, 1766.85462015, 2057.39242179, ..., 1742.99918261,
       2001.35740046, 1973.09468973])

In [ ]:
from sklearn.tree import DecisionTreeRegressor
reg_tree = DecisionTreeRegressor(min_impurity_decrease=2.0,random_state=0).fit(x1,y)
y_pred_dec=reg_tree.predict(x1)
print("RMSE train data:",mean_squared_error(y,y_pred_dec,squared=False))

RMSE train data: 122.24533514138938


In [ ]:
y_pred_dec1=reg_tree.predict(x_test1)
print("RMSE train data:",mean_squared_error(y_test,y_pred_dec1,squared=False))

RMSE train data: 126.26443876028988


In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 80)
xg_reg.fit(x1,y)
y_pred_xgb = xg_reg.predict(x1)
rmse_test=mean_squared_error(y,y_pred_xgb,squared=False)
print("RMSE of train data : ",rmse_test)

RMSE of train data :  127.01683203326021


In [ ]:
df_stack=pd.DataFrame()
df_stack['reg']=y_pred_reg.tolist()
df_stack['bas']=y_pred_bas.tolist()
df_stack['nn']=y_pred_nn.tolist()
df_stack['xgb']=y_pred_xgb.tolist()
df_stack['dec']=y_pred_dec.tolist()

In [ ]:
stack_reg=LinearRegression().fit(df_stack,y)
y_stack_pred=stack_reg.predict(df_stack)
print("RMSE train data:",mean_squared_error(y,y_stack_pred,squared=False))

RMSE train data: 119.43964571656188


In [ ]:
df_stack_test=pd.DataFrame()
df_stack_test['reg']=y_pred_reg1.tolist()
df_stack_test['bas']=reglog.predict(x_test1).tolist()
df_stack_test['nn']=preds.tolist()
df_stack_test['xgb']=xg_reg.predict(x_test1).tolist()
df_stack_test['dec']=y_pred_dec1.tolist()
y_stack_pred1=stack_reg.predict(df_stack_test)
print("RMSE train data:",mean_squared_error(y_test,y_stack_pred1,squared=False))

RMSE train data: 127.5803107255438


In [ ]:

x_human=df_tt2.drop(['rating','game_id','nickname','initial_time_seconds', 'increment_seconds'],axis=1)
x_human1=pd.DataFrame(scaler.transform(x_human),columns=cols)

In [ ]:
df_stack_human=pd.DataFrame()
df_stack_human['reg']=reg.predict(x_human1).tolist()
df_stack_human['bas']=reglog.predict(x_human1).tolist()
df_stack_human['nn']=mlp_reg.predict(x_human1).tolist()
df_stack_human['xgb']=xg_reg.predict(x_human1).tolist()
df_stack_human['dec']=reg_tree.predict(x_human1).tolist()
y_stack_human=stack_reg.predict(df_stack_human)

In [ ]:
df_sub=pd.DataFrame(columns=['game_id','rating'])
df_sub['game_id']=df_tt2['game_id']
df_sub['rating']=reg_tree.predict(x_human1).tolist()

In [ ]:
df_sub

,game_id,rating
0,2,2049.571256
3,7,1985.734875
5,11,1698.030872
7,14,1995.210209
8,27,2100.288639
...,...,...
44717,72760,2069.732249
44718,72761,1845.080070
44721,72762,1646.015214
44723,72768,2095.406095


In [ ]:
df_sub.to_csv('submission.csv',index=False)

In [ ]:
df_sub=pd.read_csv('submission.csv')

In [ ]:
df_sub

,game_id,rating
0,2,2118.558892
1,7,1996.659397
2,11,1725.711713
3,14,1980.814330
4,27,2119.878440
...,...,...
22358,72760,2089.616426
22359,72761,1665.530239
22360,72762,1655.195804
22361,72768,2041.186930
